In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 

In [2]:
codon_freq = pd.read_csv('../data/mouse_codon_freq.csv')
codon_freq =codon_freq.set_index('Unnamed: 0')

In [164]:
def compute_CAI(sequence:str, codon_freq:pd.DataFrame):
    """Compute the Codon Adaptation Index (CAI) of a given protein/mRNA sequence
  
      parameters:
      sequence: str,
          sequence of the protein/mRNA transcript of interest
      codon_freq: pd.DataFrame,
          frequencies of each codon and corresponding amino acid (species specific)

      returns:s
      CAI : foat,
          computed Codon adaptation index (CAI), returned values are between 0 and 1"""
    #Convert T to U
    sequence = sequence.translate(str.maketrans("T", "U"))
    
    L = int(len(sequence)/3)
    CAI = 0
    for i in range(0,L):
        codon = sequence[3*i:3*i+3]
        if 'N' in codon:
            continue #ignore the codon containing a N (uncertain nucleotide)
        freq = codon_freq.T[codon][1]
        aa = codon_freq.T[codon][0]
        freq_max = np.max((codon_freq[codon_freq['amino acid']==aa])['freq'])
        
        w = np.log(freq/freq_max)
        CAI += w
        
    CAI = np.exp(1/L * CAI)
    
    return CAI

In [167]:
amylase1 = 'ATGAAATTCTTCCTGCTGCTTTCCCTCATTGGATTCTGCTGGGCCCAATATGACCCACATACTCAATATGGACGAACTGCTATTGTCCACCTGTTTGAGTGGCGCTGGGTTGATATTGCTAAGGAATGTGAGAGATACTTAGCTCCTAATGGATTTGCAGGTGTGCAGGTCTCTCCACCCAATGAAAACATCGTAGTCCACAGCCCTTCAAGACCATGGTGGGAAAGATATCAACCAATTAGCTACAAAATATGTTCCAGGTCTGGAAATGAAGATGAATTCAGGGACATGGTGAACAGGTGCAACAATGTTGGTGTCCGTATTTATGTGGATGCTGTCATTAACCACATGTGTGGAGTGGGGGCTCAAGCTGGACAAAGCAGTACATGTGGAAGTTATTTCAACCCAAATAACAGGGACTTTCCTGGAGTTCCCTATTCTGGTTTTGACTTTAATGATGGAAAATGTAGAACTGCAAGTGGAGGTATCGAGAACTACCAAGATGCTGCTCAGGTCAGAGATTGTCGTCTGTCTGGCCTTCTGGATCTTGCACTTGAGAAAGATTATGTTCGAACCAAGGTGGCTGACTATATGAACCATCTCATTGACATTGGCGTAGCAGGGTTCAGACTTGATGCTTCTAAGCACATGTGGCCTGGAGACATAAAGGCAATTTTGGACAAACTGCATAATCTCAATACAAAATGGTTCTCCCAAGGAAGCAGACCTTTCATTTTCCAAGAGGTGATTGATCTGGGTGGTGAGGCAGTGTCAAGTAATGAGTATTTTGGAAATGGCCGTGTGACAGAATTCAAATATGGAGCAAAATTGGGCAAAGTTATGCGCAAGTGGGATGGAGAAAAGATGTCCTACTTAAAGAACTGGGGAGAAGGTTGGGGTTTGATGCCTTCTGACAGAGCCCTTGTGTTTGTGGACAACCATGACAATCAGCGAGGACATGGTGCTGGGGGAGCATCCATCTTGACATTCTGGGATGCTAGACTCTATAAAATGGCTGTTGGCTTTATGTTGGCTCATCCTTATGGTTTCACACGGGTGATGTCAAGTTACTATTGGCCAAGAAATTTCCAGAATGGAAAAGATGTCAATGACTGGGTTGGACCACCAAATAACAATGGAAAAACCAAAGAAGTGAGCATTAACCCAGACAGCACTTGTGGCAATGACTGGATCTGTGAACATCGATGGCGTCAAATAAGGAACATGGTTGCCTTCAGAAATGTCGTCAATGGTCAGCCTTTTGCAAACTGGTGGGATAATGACAGCAACCAGGTAGCTTTTGGCAGAGGAAACAAAGGATTCATTGTCTTTAACAATGATGACTGGGCTTTGTCAGAAACTTTACAGACTGGTCTTCCTGCTGGCACATACTGTGATGTCATTTCTGGAGATAAAGTCGATGGCAATTGCACTGGAATAAAAGTCTATGTTGGCAATGATGGCAAAGCTCACTTTTCTATTAGTAACTCTGCCGAAGACCCATTTATTGCAATCCATGCAGAGTCAAAAATATAA'
compute_CAI(amylase1,codon_freq)

0.7578865327134019

In [5]:
def read_fasta(fasta_file):
    "Parse fasta file"
    count = 0
    headers = []
    sequences = []
    aux = []
    with open(fasta_file, 'r') as infile:
        for line in infile:
            record = line.rstrip()
            if record and record[0] == '>':
                headers.append(record[1:])
                if count > 0:
                    sequences.append(''.join(aux))
                    aux = []
            else:
                aux.append(record)
            count += 1

    sequences.append(''.join(aux))
    return headers, sequences 

In [163]:
#Load all the coding sequence from the file
genes, cds = read_fasta('../data/mart_export.txt')
for i, gene in enumerate(genes):
    genes[i] = (gene).split('|')[0]
genes = pd.DataFrame(cds, index = genes, columns = ['cds'])
genes = genes[~genes.index.duplicated()]

#Only keep the one that are in the AE3/AE4 datasets
AE3 = pd.read_csv ('../data/merged_data/AE3.csv')
AE3 = AE3.set_index('Unnamed: 0')
genes = genes.loc[np.unique(AE3.index.intersection(genes.index))]

#Remove not defined sequence genes
error = genes.iloc[3][0]
genes = genes[genes.cds != error]

genes.to_csv('../data/genes.csv', index=True)

In [ ]:
#Compute the CAI for each gene
CAI = []
for cds in genes['cds']:
    CAI.append(compute_CAI(cds,codon_freq))
    
genes['CAI'] = CAI

In [ ]:
genes.to_csv('../data/CAI.csv', index=True)